In [3]:
from sklearn.linear_model import Ridge
import joblib
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [4]:
data_oof = pd.read_csv("../data/df_train_oof.csv", index_col=0)  # 加载分层5折数据集
data_oof.head()

,id,url_legal,license,excerpt,target,standard_error,fold
0,b51730f9c,NaN,NaN,"Alice looked at the jury-box, and saw that, in...",-0.432678,0.487498,0
1,4d403fd57,https://en.wikipedia.org/wiki/Artificial_intel...,CC BY-SA 3.0,Artificial intelligence (AI) is intelligence e...,-1.161746,0.458396,2
2,0f789ee41,NaN,NaN,A gruff squire on horseback with shiny top boo...,-2.367914,0.519369,4
3,87f96eb79,NaN,NaN,But that hadn't helped Washington.\nThe Americ...,-0.842596,0.466193,0
4,b9cca6661,NaN,NaN,The principal business of the people of this c...,-0.748452,0.433000,4


In [5]:
y_train = pd.concat([data_oof[data_oof['fold'] == 0]['target'], 
                     data_oof[data_oof['fold'] == 1]['target'], 
                     data_oof[data_oof['fold'] == 2]['target'],
                     data_oof[data_oof['fold'] == 3]['target'],
                     data_oof[data_oof['fold'] == 4]['target']]).values
y_train

array([-0.43267753, -0.84259602, -2.40380493, ..., -2.13429795,
       -1.2519907 , -0.6052157 ])

In [6]:
X_train_roberta = joblib.load('roberta_mlm_cv_train/roberta_large_tr.pkl')
X_train_alberta = joblib.load('albert_mlm_cv_train/albert_xxlarge_v2_tr.pkl')
X_train_electra = joblib.load('electra_cv_train/electra_large_tr.pkl')
X_train_stacking = np.concatenate((X_train_roberta, X_train_alberta, X_train_electra), axis=1)
X_train_stacking.shape

(2834, 3)

In [7]:
X_train_stacking, y_train = shuffle(X_train_stacking, y_train, random_state=42) 

In [8]:
ridge_reg = Ridge(alpha=0.001, max_iter=1000)  # 设置alpha=0.001(Regularization strength)
ridge_reg.fit(X_train_stacking, y_train)

Ridge(alpha=0.001, max_iter=1000)

In [9]:
X_test_roberta = joblib.load('roberta_mlm_cv_train/roberta_large_te.pkl').reshape(7, -1).mean(axis=1)
X_test_alberta = joblib.load('albert_mlm_cv_train/albert_xxlarge_v2_te.pkl').reshape(7, -1).mean(axis=1)
X_test_electra = joblib.load('electra_cv_train/electra_large_te.pkl').reshape(7, -1).mean(axis=1)
X_test_stacking = np.stack((X_test_roberta, X_test_alberta, X_test_electra), axis=1)
X_test_stacking

array([[-1.2130673 , -1.2349802 , -1.0639806 ],
       [-0.48608837, -0.51714957, -0.56315154],
       [-1.1838045 , -1.0915611 , -1.0860678 ],
       [-1.3047185 , -1.3212792 , -1.2881937 ],
       [-0.64509046, -0.65288514, -0.6494516 ],
       [-0.87645596, -0.91668445, -0.83477944],
       [-1.1311338 , -1.2475598 , -1.1726393 ]], dtype=float32)

In [10]:
ridge_reg.predict(X_test_stacking)  # stacking模型融合

array([-1.167804  , -0.5094876 , -1.1230512 , -1.3069384 , -0.6390131 ,
       -0.86803937, -1.1795855 ], dtype=float32)

In [11]:
X_test_roberta * 0.4 + X_test_alberta * 0.3 + X_test_electra * 0.3  # 线性加权融合(可通过optuna包搜索各模型权重(为超参数))

array([-1.1749152 , -0.5185257 , -1.1268104 , -1.3047293 , -0.64873725,
       -0.8760215 , -1.1785133 ], dtype=float32)